In [2]:
#ОБРУЧАЛКА

import pandas as pd

# Загужаем файл остатков и отметает лишнее

ost_obr = pd.read_csv(r'C:\анаконда\tmp.csv', delimiter=';', encoding='utf-8',low_memory=False,
               usecols=["КодСклада", "НазваниеСклада", "ТоварНо","СерийныйНомер","Проект"])

ost_obr = ost_obr[ost_obr.Проект.isin(['ЗОЛОТОЙ','ОРИОН','ЗОЛОТО','ТАЛАНТ', 'НОУНЕЙМ'])].sort_values('КодСклада')

del ost_obr['Проект']

# Делим файл на две части:магазы и центральные склады

ost_obr1 = ost_obr[ost_obr.НазваниеСклада.str.contains('центральный склад|опт',case = False)]
ost_obr = ost_obr[~ost_obr.НазваниеСклада.str.contains('центральный склад|опт',case = False)]
ost_obr = ost_obr[['КодСклада','ТоварНо']]
ost_obr1 = ost_obr1[['СерийныйНомер','ТоварНо']]

# Обработка заборников верхние пустые строки в загружаемом файле должны быть удалены

zab = pd.read_excel(r'C:\анаконда\zab.xlsx', skiprows=4)
zab = zab[["Серийный Но.","Товар Но.","Склад-приемник","Номер ЗЛ"]].sort_values(by="Номер ЗЛ",ascending = False)
zab.columns = ['СерийныйНомер','ТоварНо','КодСклада','ЗЛ']
zab.rename(
    columns={"Серийный Но.":'СерийныйНомер', "Товар Но.":'ТоварНо', 
             "Склад-приемник":'КодСклада',"Номер ЗЛ":'ЗЛ'}, 
    inplace=True
) 

# Делим файл на две части:необиркованные и без СН на машине

zab_0 = zab[zab.СерийныйНомер.isnull()]
zab_0 = zab_0[['КодСклада','ТоварНо']]
zab = zab[zab.СерийныйНомер.str.contains('СН',na=False)]
zab = zab[['СерийныйНомер','КодСклада']]

# Проставляем магаз в серийники, которые на 4001/3075/опте

df = pd.merge(ost_obr1,zab,on=['СерийныйНомер'],how = 'inner')#inner join оставит только неперемещенный товар
ost_obr1 = df[['КодСклада','ТоварНо']]

# Соединяем остатки на магазах,на машине, то что числилось на 4001/3075/опте
result = pd.concat([ost_obr,ost_obr1,zab_0])

# Догружаем размер/артикул/толщ.плетения и тд

df = pd.read_excel(r'C:\анаконда\карточки.xlsx',sheet_name='обручи',header=6)
result = pd.merge(result,df,left_on=['ТоварНо'],right_on=['Номер'],how='left')
del result['Номер']
result.set_index('КодСклада',drop=True, inplace=True)

# Сохраняем в эксель. нужно доработать сохранение в уже существующий эксель

with pd.ExcelWriter(r'C:\Остатки\Обручальные кольца.xlsx') as writer:
    result.to_excel(writer, sheet_name='остатки')

In [1]:
#БУКВЫ И ЗОДИАКИ

import pandas as pd

# Загужаем файл остатков и отметает лишнее

ost_obr = pd.read_csv(r'C:\анаконда\tmp.csv', delimiter=';', encoding='utf-8',low_memory=False,
               usecols=["КодСклада", "НазваниеСклада", "ТоварНо","СерийныйНомер","ТоварноеНаправление","Проект","Тип1"])

# Оставляем только декоративные подвесы и нужные проекты

ost_obr = ost_obr[(ost_obr["Тип1"] == 'ПОДВЕС ДЕКОРАТИВНЫЙ')]
ost_obr = ost_obr[ost_obr.Проект.isin(['ЗОЛОТОЙ','ОРИОН','ЗОЛОТО','ТАЛАНТ', 'НОУНЕЙМ'])].sort_values('КодСклада')
del ost_obr['Проект'], ost_obr['Тип1']

# Делим файл на две части:магазы и центральные склады

ost_obr1 = ost_obr[ost_obr.НазваниеСклада.str.contains('центральный склад|опт',case = False)]
ost_obr = ost_obr[~ost_obr.НазваниеСклада.str.contains('центральный склад|опт',case = False)]
ost_obr = ost_obr[['КодСклада','ТоварНо','ТоварноеНаправление']]
ost_obr1 = ost_obr1[['СерийныйНомер','ТоварНо','ТоварноеНаправление']]

# Обработка заборников верхние пустые строки в загружаемом файле должны быть удалены

zab = pd.read_excel(r'C:\анаконда\zab.xlsx',
                  usecols=["Серийный Но.","Товар Но.","Склад-приемник","Номер ЗЛ","Товарное направление","Тип 1"], skiprows=4)

zab = zab[(zab["Тип 1"] == 'ПОДВЕС ДЕКОРАТИВНЫЙ')]
del zab["Тип 1"]
zab = zab.sort_values(by="Номер ЗЛ",ascending = False)

zab.rename(
    columns={"Серийный Но.":'СерийныйНомер', "Товар Но.":'ТоварНо', 
             "Склад-приемник":'КодСклада',"Номер ЗЛ":'ЗЛ',"Товарное направление":'ТоварноеНаправление' }, 
    inplace=True
) 

# Делим файл на две части:необиркованные и без СН на машине

zab_0 = zab[zab.СерийныйНомер.isnull()]
zab_0 = zab_0[['КодСклада','ТоварНо','ТоварноеНаправление']]
zab = zab[zab.СерийныйНомер.str.contains('СН',na=False)]
zab = zab[['СерийныйНомер','КодСклада']]

# Проставляем магаз в серийники, которые на 4001/3075/опте

df = pd.merge(ost_obr1,zab,on=['СерийныйНомер'],how = 'inner')#inner join оставит только неперемещенный товар
ost_obr1 = df[['КодСклада','ТоварНо','ТоварноеНаправление']]

# Соединяем остатки на магазах,на машине, то что числилось на 4001/3075/опте
result = pd.concat([ost_obr,ost_obr1,zab_0])


# Догружаем размер/артикул/толщ.плетения и тд

df = pd.read_excel(r'C:\анаконда\карточки.xlsx',sheet_name='буки',header=6)
result = pd.merge(result,df,left_on=['ТоварНо'],right_on=['Номер'],how='left')
del result['Номер']
result.set_index('КодСклада',drop=True, inplace=True)

# Очищаем описание 3 от ненужного мусора
result['Описание 3'] = result['Описание 3'].apply(lambda a: a[:str(a).find(' ')] if str(a).find(' ')>0 else a )


# Сохраняем в эксель. нужно доработать сохранение в уже существующий эксель

with pd.ExcelWriter(r'C:\Остатки\буквыизодиаки.xlsx') as writer:
    result.to_excel(writer)
    

In [7]:
# Тип1 направление
import pandas as pd
#загужаем файл остатков и отметает лишнее
ost_obr = pd.read_csv(r'C:\анаконда\tmp.csv', delimiter=';', encoding='utf-8',low_memory=False,
                    usecols=["КодСклада", "НазваниеСклада", "Тип1",
                                "ТоварноеНаправление","ТоварНо","СерийныйНомер","Проект"])
ost_obr = ost_obr[ost_obr.Проект.isin(['ЗОЛОТОЙ','ОРИОН','ЗОЛОТО','ТАЛАНТ', 'НОУНЕЙМ'])].sort_values('КодСклада')
del ost_obr['Проект']

#делим остатки на две части:магазы и центральные склады (то что на центральных складах может быть уже распределено)
ost_obr1 = ost_obr[ost_obr.НазваниеСклада.str.contains('центральный склад|опт',case = False)]
ost_obr = ost_obr[~ost_obr.НазваниеСклада.str.contains('центральный склад|опт',case = False)]
del ost_obr1['КодСклада']

#обработка заборников верхние пустые строки в загружаемом файле должны быть удалены
zab = pd.read_excel(r'C:\анаконда\zab.xlsx', skiprows=4)
zab = zab[["Серийный Но.","Склад-приемник","Номер ЗЛ"]].sort_values(by="Номер ЗЛ",ascending = False)
zab.columns = ['СерийныйНомер','КодСклада','ЗЛ']
zab.rename(columns={"Серийный Но.":'СерийныйНомер', "Склад-приемник":'КодСклада',"Номер ЗЛ":'ЗЛ'}, inplace=True) 
del zab['ЗЛ']

# из заборных листов проставляем номер склада для серийников, которые на остатке 4001/3075/опте
ost_obr1 = pd.merge(ost_obr1,zab,on=['СерийныйНомер'],how = 'inner')#inner join оставит только неперемещенный товар
ost_obr1 = ost_obr1[["КодСклада", "НазваниеСклада", "ТоварНо","СерийныйНомер","Тип1","ТоварноеНаправление"]]

#соединяем остатки на магазах,на машине, то что числилось на 4001/3075/опте
result = pd.concat([ost_obr,ost_obr1])
result.set_index('КодСклада',drop=True, inplace=True)
del result['НазваниеСклада']

#сохраняем в эксель. нужно доработать сохранение в уже существующий эксель
with pd.ExcelWriter(r'C:\Остатки\остатки.xlsx') as writer:
    result.to_excel(writer)

In [2]:
#ЛЮБОЙ ТОВАР

import pandas as pd
# Загружаем файл остатков и отметает лишнее

ost_obr = pd.read_csv(r'C:\анаконда\tmp.csv', delimiter=';', encoding='utf-8',low_memory=False,
                    usecols=["КодСклада", "НазваниеСклада", "ТоварНо","СерийныйНомер","Проект"])

ost_obr = ost_obr[ost_obr.Проект.isin(['ЗОЛОТОЙ','ОРИОН','ЗОЛОТО','ТАЛАНТ', 'НОУНЕЙМ'])].sort_values('КодСклада')
del ost_obr['Проект']

# Делим файл на две части:магазы и центральные склады

ost_obr1 = ost_obr[ost_obr.НазваниеСклада.str.contains('центральный склад|опт',case = False)]
ost_obr = ost_obr[~ost_obr.НазваниеСклада.str.contains('центральный склад|опт',case = False)]
ost_obr = ost_obr[["КодСклада", "ТоварНо", "СерийныйНомер"]]
del ost_obr1['КодСклада']

# обработка заборников 

zab = pd.read_excel(r'C:\анаконда\zab.xlsx', skiprows=4)
zab = zab[["Серийный Но.","Склад-приемник","Номер ЗЛ"]].sort_values(by="Номер ЗЛ",ascending = False)
zab.rename(columns={"Серийный Но.":'СерийныйНомер', "Склад-приемник":'КодСклада',"Номер ЗЛ":'ЗЛ'}, inplace=True) 
del zab['ЗЛ']

# проставляем магаз в серийники, которые на 4001/3075/опте

ost_obr1 = pd.merge(ost_obr1,zab,on=['СерийныйНомер'],how = 'inner')#inner join оставит только неперемещенный товар
ost_obr1 = ost_obr1[["КодСклада",  "ТоварНо","СерийныйНомер"]]

# соединяем остатки на магазах,на машине, то что числилось на 4001/3075/опте

result = pd.concat([ost_obr,ost_obr1])
result.set_index('КодСклада',drop=True, inplace=True)

# сохраняем в эксель. нужно доработать сохранение в уже существующий эксель

with pd.ExcelWriter(r'C:\Остатки\остатки.xlsx') as writer:
    result.to_excel(writer)

In [1]:
# колонки на выбор
import pandas as pd
# Columns to use/ first four are obligatory

"""    'Тип1', 'Тип2', 'Тип3', 'Дизайн', 'Производитель', 'Поставщик', 'ПоставщикАрт',
       'ШК', 'СтарыйЯрлык', 'КолВо', 'Вес', 'ЧистыйВес', 'Проба1',
       'РазмерИзделия', 'ВставкаКамней', 'ТолщинаПлетения', 'КолвоКамней',
       'СуммаРеализации', 'СуммаПрихода', 'ДатаПартии', 'ДатаПрихода',
       'ДатаВТорг', 'ОписаниеТовара', 'ГруппаНаценки', 'ТипИзделия',
       'СкидкаПоШК', 'СкидакаПоГруппе', 'СкидкаЗапрещена', 'ТоварнаяПодгруппа',
       'ПереоцененнаяСтоимость', 'ТоварноеНаправление', 'Картинка', 'Проект',
       'Невыкупа', 'ЛогистическийПризнак', 'ТоварнаяГруппа', 'Price_f',
       'Gender', 'ПроизводитАртКартТов'"""

##################################################################################################
cols = ["КодСклада",'НазваниеСклада',"ТоварНо","СерийныйНомер", 'ТоварноеНаправление',  'Тип1', 'ТоварнаяГруппа']
#################################################################################################загужаем файл остатков и отметает лишнее
ost_obr = pd.read_csv(r'C:\анаконда\tmp.csv', delimiter=';', encoding='utf-8',low_memory=False,
                    usecols=cols + ['Проект'] )
ost_obr = ost_obr[ost_obr.Проект.isin(['ЗОЛОТОЙ','ОРИОН','ЗОЛОТО','ТАЛАНТ', 'НОУНЕЙМ'])].sort_values('КодСклада')
del ost_obr['Проект']

#делим остатки на две части:магазы и центральные склады (то что на центральных складах может быть уже распределено)
ost_obr1 = ost_obr[ost_obr.НазваниеСклада.str.contains('центральный склад|опт',case = False)]
ost_obr = ost_obr[~ost_obr.НазваниеСклада.str.contains('центральный склад|опт',case = False)]
del ost_obr1['КодСклада']

#обработка заборников верхние пустые строки в загружаемом файле должны быть удалены
zab = pd.read_excel(r'C:\анаконда\zab.xlsx', skiprows=4)
zab = zab[["Серийный Но.","Склад-приемник","Номер ЗЛ"]].sort_values(by="Номер ЗЛ",ascending = False)
zab.columns = ['СерийныйНомер','КодСклада','ЗЛ']
zab.rename(columns={"Серийный Но.":'СерийныйНомер', "Склад-приемник":'КодСклада',"Номер ЗЛ":'ЗЛ'}, inplace=True) 
del zab['ЗЛ']

# из заборных листов проставляем номер склада для серийников, которые на остатке 4001/3075/опте
ost_obr1 = pd.merge(ost_obr1,zab,on=['СерийныйНомер'],how = 'inner')#inner join оставит только неперемещенный товар
ost_obr1 = ost_obr1[cols]

#соединяем остатки на магазах,на машине, то что числилось на 4001/3075/опте
result = pd.concat([ost_obr,ost_obr1])
result.set_index('КодСклада',drop=True, inplace=True)
del result['НазваниеСклада']


type1_lst = result['Тип1'].unique()[:8]

path = 'C:/Остатки'
for type in type1_lst:
    result[result['Тип1'] == type].to_excel(path + '/' + type + '.xlsx')


In [3]:
#ЛЮБОЙ ТОВАР 5442

import pandas as pd
# Загружаем файл остатков и отметает лишнее
lst = [5442, 5455]
ost_obr = pd.read_csv(r'C:\анаконда\tmp.csv', delimiter=';', encoding='utf-8',low_memory=False,
                    usecols=["КодСклада", "НазваниеСклада", "ТоварНо","СерийныйНомер","Проект"])

ost_obr = ost_obr[ost_obr.Проект.isin(['ЗОЛОТОЙ','ОРИОН','ЗОЛОТО','ТАЛАНТ', 'НОУНЕЙМ'])].sort_values('КодСклада')
del ost_obr['Проект']

# Делим файл на две части:магазы и центральные склады

ost_obr1 = ost_obr[ost_obr.НазваниеСклада.str.contains('центральный склад|опт',case = False)]
ost_obr = ost_obr[ost_obr.КодСклада.isin(lst)]
ost_obr = ost_obr[["КодСклада", "ТоварНо", "СерийныйНомер"]]
del ost_obr1['КодСклада']

# обработка заборников 

zab = pd.read_excel(r'C:\анаконда\zab.xlsx', skiprows=4)
zab = zab[["Серийный Но.","Склад-приемник","Номер ЗЛ"]].sort_values(by="Номер ЗЛ",ascending = False)
zab = zab[zab['Склад-приемник'].isin(lst)]
zab.rename(columns={"Серийный Но.":'СерийныйНомер', "Склад-приемник":'КодСклада',"Номер ЗЛ":'ЗЛ'}, inplace=True) 
del zab['ЗЛ']

# проставляем магаз в серийники, которые на 4001/3075/опте

ost_obr1 = pd.merge(ost_obr1,zab,on=['СерийныйНомер'],how = 'inner')#inner join оставит только неперемещенный товар
ost_obr1 = ost_obr1[["КодСклада",  "ТоварНо","СерийныйНомер"]]

# соединяем остатки на магазах,на машине, то что числилось на 4001/3075/опте

result = pd.concat([ost_obr,ost_obr1])
result.set_index('КодСклада',drop=True, inplace=True)

# сохраняем в эксель. нужно доработать сохранение в уже существующий эксель

with pd.ExcelWriter(r'C:\Остатки\остатки.xlsx') as writer:
    result.to_excel(writer)


In [31]:
import pandas as pd

df = pd.read_csv(r'C:\анаконда\tmp.csv', delimiter=';', encoding='utf-8',low_memory=False,
               usecols=['ПоставщикАрт', "ТоварНо", 'ГруппаНаценки'])

In [41]:
sum(df['ГруппаНаценки'].str.contains('ИФ.БК') == False)


276654

In [43]:
df[df["ТоварНо"].isin(['ТОВ0514704', 'ТОВ1095537'])]

,ТоварНо,ПоставщикАрт,ГруппаНаценки
1536,ТОВ1095537,12-02-0001-26411,ИФ_ПУССЕТЫ
4986,ТОВ1095537,12-02-0001-26411,ИФ_ПУССЕТЫ
6581,ТОВ1095537,12-02-0001-26411,ИФ_ПУССЕТЫ
8512,ТОВ1095537,12-02-0001-26411,ИФ_ПУССЕТЫ
23820,ТОВ1095537,12-02-0001-26411,ИФ_ПУССЕТЫ
...,...,...,...
2144534,ТОВ1095537,12-02-0001-26411,ИФ_ПУССЕТЫ
2144899,ТОВ1095537,12-02-0001-26411,ИФ_ПУССЕТЫ
2146753,ТОВ1095537,12-02-0001-26411,ИФ_ПУССЕТЫ
2151946,ТОВ1095537,12-02-0001-26411,ИФ_ПУССЕТЫ


In [6]:
lst = pd.read_excel('C:/рушники/123.xlsx', sheet_name='Лист1')

In [5]:
df.drop_duplicates(inplace=True)

In [17]:
arts = lst['Артикул товара','Изменения']][lst['Изменения '] != 'НЕТ'].tolist()

In [28]:
df[df['ПоставщикАрт'].isin(arts)].to_excel(r'C:\Остатки\123.xlsx')


In [25]:
df

,ТоварНо,ПоставщикАрт
0,ТОВ0690947,20-02-0001-20230
1,ТОВ0691249,12.00_15
2,ТОВ1051314,10-03-1629
3,ТОВ1051340,КФ 193#БЕЛ
4,ТОВ1051351,КСК 228#БЕЛ
...,...,...
2147359,ТОВ0855440,543
2147445,ТОВ0727541,К120164
2147542,ТОВ1045205,С010-01-09
2147612,ТОВ0688433,330


In [24]:
lst

,Товарное направление,Группа наценки,Артикул товара,Средний вес изделия,Изменения
0,БК,NaN,NaN,0.00,ИФ.БК_ДЕШ
1,БК,БК_БИМЕТАЛЛ_СР,SHY00099,0.00,НЕТ
2,БК,БК_БИМЕТАЛЛ_СР,SHY00088,0.00,НЕТ
3,БК,БК_БИМЕТАЛЛ_СР,YUZ187,0.00,НЕТ
4,БК,БК_БИМЕТАЛЛ_СР,YUZ202,0.00,НЕТ
...,...,...,...,...,...
9732,ИФ,ОПТ_ИФ,СФ 202#5#ТД,2.91,НЕТ
9733,ИФ,ОПТ_ИФ,СФ 202#6#ТД,2.75,НЕТ
9734,ИФ,ОПТ_ИФ,СФ 553#ТД,1.76,НЕТ
9735,ИФ,ОПТ_ИФ,СФ 569#ТД,1.79,НЕТ
